#Intro to some of the more peculiar aspects of Julia
##... Introspection, Types, Multiple Dispatch, Metaprogramming

Julia Documentation: http://julia.readthedocs.org/en/latest/manual/introduction/

##Introspection

Most of julia is written in itself. This means, that the best resource for learning how julia works is it's source code.
To help you understand it a little bit better, and to make the source code accessible to you, Julia comes packed with a set of tools for introspection.

####Access the docstrings:
```julia

? func_name

```

####Some helpful functions related to introspection:


In [22]:
? @which

Base.@which()

   Evaluates the arguments to the function call, determines their
   types, and calls the "which" function on the resulting expression


In [23]:
@which 2+2

+(x::Int64,y::Int64) at int.jl:12

In [17]:
? methods

Base.methods(f[, types])

   Show all methods of "f" with their argument types.

   If "types" is specified, an array of methods whose types match is
   returned.


#Typesystem
Julia has a rich type system and actually cares quite a lot about the types passed to arguments. This is aprt of their recipe to achieve speed. Functions can be optimised to meet the demands of specific combinations of input-argument-types. *->Multiple Dispatch, more on that later*

To render specification of such functions, which are tailored to specific types more convenient, Julia implements a tree-like type hierarchy. **Concrete types** are the endpoint of branches in this tree, which in turn are specified by **abstract types**.

Julia's type tree: https://gist.githubusercontent.com/tanmaykm/5088310/raw/julia_type_tree.txt

In [32]:
? typeof

Base.typeof(x)

   Get the concrete type of "x".


In [35]:
typeof(2)

Int64

###Subtypes and Supertypes

In [39]:
? <:

Base.issubtype(type1, type2)

   True if and only if all values of "type1" are also of "type2".
   Can also be written using the "<:" infix operator as "type1 <:
   type2".


In [42]:
typeof(2) <: Number

true

In [45]:
typeof(2) <: Integer

true

In [47]:
? super

Base.super(T::DataType)

   Return the supertype of DataType T


In [50]:
super(Int64)

Signed

In [52]:
super(Signed)

Integer

In [55]:
super(Integer)

Real

In [58]:
super(Real)

Number

**The Root of the type system**

In [69]:
super(Number)

Any

In [70]:
super(Any)

Any

###Exercise: Traverse the Type-Tree
Write a function that returns all the abstract supertypes of a given concrete type.

In [66]:
function squirrel(x::Type)
    s = super(x)
    println(s)
    if s != Any
        squirrel(s)
    end
end

squirrel (generic function with 1 method)

In [68]:
squirrel(Float64)

FloatingPoint
Real
Number
Any


###User defined types

In [78]:
#a new branch

abstract myBranch <: Any
abstract myTwig <: myBranch

In [80]:
# a new leaf
type Leaf <: myTwig
    color::String
    shape::Integer
end

LoadError: invalid redefinition of constant Leaf
while loading In[80], in expression starting on line 2

In [76]:
Leaf("green", 5)

Leaf("green",5)

In [77]:
squirrel(Leaf)

myBranch
Any


In [20]:
? names

Base.names(x::Module[, all=false[, imported=false]])

   Get an array of the names exported by a module, with optionally
   more module globals according to the additional parameters.

Base.names(x::DataType)

   Get an array of the fields of a data type.


In [1]:
names(Leaf)

LoadError: Leaf not defined
while loading In[1], in expression starting on line 1

In [82]:
#parametric types: Declaring a vast number of types at once
type Measure{T}
    qty::T
    unit::String
end

LoadError: invalid redefinition of constant Measure
while loading In[82], in expression starting on line 2

In [85]:
length = Measure{Float64}(20., "m")

Measure{Float64}(20.0,"m")

In [86]:
imaginaryLeafMeasure = Measure{Leaf}(Leaf("teal", 20), "meaninglessLeafUnit")

Measure{Leaf}(Leaf("teal",20),"meaninglessLeafUnit")


#Multiple Dispatch

https://en.wikipedia.org/wiki/Multiple_dispatch


Functions can behave differently based on the types of their input arguments. Think: operator overloading!



In [14]:
@which 2*2

*(x::Int64,y::Int64) at int.jl:14

In [15]:
@which "a"*"b"

*(s::AbstractString...) at string.jl:76

###Type Conversion

In [10]:
? convert

Base.convert(T, x)

   Convert "x" to a value of type "T".

   If "T" is an "Integer" type, an "InexactError" will be raised
   if "x" is not representable by "T", for example if "x" is not
   integer-valued, or is outside the range supported by "T".

      julia> convert(Int, 3.0)
      3

      julia> convert(Int, 3.5)
      ERROR: InexactError()
       in convert at int.jl:185

   If "T" is a "FloatingPoint" or "Rational" type, then it will
   return the closest value to "x" representable by "T".

      julia> x = 1/3
      0.3333333333333333

      julia> convert(Float32, x)
      0.33333334f0

      julia> convert(Rational{Int32}, x)
      1//3

      julia> convert(Rational{Int64}, x)
      6004799503160661//18014398509481984


In [24]:
methods(convert)

# 488 methods for generic function "convert":
convert(::(),::()) at base.jl:38
convert(::Type{(Any...,)},x::(Any...,)) at base.jl:39
convert{T}(::Type{(T...,)},x::(Any...,)) at base.jl:49
convert(T::(Type{T<:Top},Type{T<:Top}...),x::(Any,Any...)) at base.jl:44
convert{T}(::Type{FloatRange{T}},r::FloatRange{T<:FloatingPoint}) at range.jl:468
convert{T}(::Type{FloatRange{T}},r::OrdinalRange{T,S}) at range.jl:473
convert{T}(::Type{Array{T,1}},r::Range{T}) at range.jl:508
convert(::Type{Bool},x::Bool) at bool.jl:3
convert(::Type{Bool},x::Rational{T<:Integer}) at rational.jl:64
convert(::Type{Bool},x::Float16) at float16.jl:99
convert(::Type{Bool},x::Base.MPFR.BigFloat) at mpfr.jl:154
convert(::Type{Bool},x::Real) at bool.jl:4
convert(::Type{Int8},x::UInt8) at int.jl:169
convert(::Type{Int8},x::Int16) at int.jl:154
convert(::Type{Int8},x::UInt16) at int.jl:154
convert(::Type{Int8},x::Int32) at int.jl:154
convert(::Type{Int8},x::UInt32) at int.jl:154
convert(::Type{Int8},x::Int64) at int.jl:154
convert(::Type{Int8},x::UInt64) at int.jl:154
convert(::Type{Int8},x::Int128) at int.jl:154
convert(::Type{Int8},x::UInt128) at int.jl:154
convert(::Type{Int8},x::Char) at int.jl:154
convert(::Type{Int8},x::Bool) at int.jl:163
convert(::Type{UInt8},x::Int8) at int.jl:169
convert(::Type{UInt8},x::Int16) at int.jl:156
convert(::Type{UInt8},x::UInt16) at int.jl:156
convert(::Type{UInt8},x::Int32) at int.jl:156
convert(::Type{UInt8},x::UInt32) at int.jl:156
convert(::Type{UInt8},x::Int64) at int.jl:156
convert(::Type{UInt8},x::UInt64) at int.jl:156
convert(::Type{UInt8},x::Int128) at int.jl:156
convert(::Type{UInt8},x::UInt128) at int.jl:156
convert(::Type{UInt8},x::Char) at int.jl:156
convert(::Type{UInt8},x::Bool) at int.jl:163
convert(::Type{Int16},x::Int8) at int.jl:161
convert(::Type{Int16},x::UInt8) at int.jl:163
convert(::Type{Int16},x::UInt16) at int.jl:169
convert(::Type{Int16},x::Int32) at int.jl:154
convert(::Type{Int16},x::UInt32) at int.jl:154
convert(::Type{Int16},x::Int64) at int.jl:154
convert(::Type{Int16},x::UInt64) at int.jl:154
convert(::Type{Int16},x::Int128) at int.jl:154
convert(::Type{Int16},x::UInt128) at int.jl:154
convert(::Type{Int16},x::Char) at int.jl:154
convert(::Type{Int16},x::Bool) at int.jl:163
convert(::Type{UInt16},x::Int8) at int.jl:161
convert(::Type{UInt16},x::UInt8) at int.jl:163
convert(::Type{UInt16},x::Int16) at int.jl:169
convert(::Type{UInt16},x::Int32) at int.jl:156
convert(::Type{UInt16},x::UInt32) at int.jl:156
convert(::Type{UInt16},x::Int64) at int.jl:156
convert(::Type{UInt16},x::UInt64) at int.jl:156
convert(::Type{UInt16},x::Int128) at int.jl:156
convert(::Type{UInt16},x::UInt128) at int.jl:156
convert(::Type{UInt16},x::Char) at int.jl:156
convert(::Type{UInt16},x::Bool) at int.jl:163
convert(::Type{Int32},x::Int8) at int.jl:161
convert(::Type{Int32},x::UInt8) at int.jl:163
convert(::Type{Int32},x::Int16) at int.jl:161
convert(::Type{Int32},x::UInt16) at int.jl:163
convert(::Type{Int32},x::UInt32) at int.jl:169
convert(::Type{Int32},x::Int64) at int.jl:154
convert(::Type{Int32},x::UInt64) at int.jl:154
convert(::Type{Int32},x::Int128) at int.jl:154
convert(::Type{Int32},x::UInt128) at int.jl:154
convert(::Type{Int32},x::Char) at int.jl:169
convert(::Type{Int32},x::Bool) at int.jl:163
convert(::Type{UInt32},x::Int8) at int.jl:161
convert(::Type{UInt32},x::UInt8) at int.jl:163
convert(::Type{UInt32},x::Int16) at int.jl:161
convert(::Type{UInt32},x::UInt16) at int.jl:163
convert(::Type{UInt32},x::Int32) at int.jl:169
convert(::Type{UInt32},x::Int64) at int.jl:156
convert(::Type{UInt32},x::UInt64) at int.jl:156
convert(::Type{UInt32},x::Int128) at int.jl:156
convert(::Type{UInt32},x::UInt128) at int.jl:156
convert(::Type{UInt32},x::Char) at int.jl:171
convert(::Type{UInt32},x::Bool) at int.jl:163
convert(::Type{Int64},x::Int8) at int.jl:161
convert(::Type{Int64},x::UInt8) at int.jl:163
convert(::Type{Int64},x::Int16) at int.jl:161
convert(::Type{Int64},x::UInt16) at int.jl:163
convert(::Type{Int64